# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [74]:
from google.colab import userdata
userdata.get('HF_TOKEN')

NotebookAccessError: Notebook does not have access to secret HF_TOKEN

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-ar0j2g2x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-ar0j2g2x
  Resolved https://github.com/huggingface/accelerate.git to commit f0b030554cbcd01c5541c449e92066715f21a99e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.2.0.dev0-py3-none-any.whl size=336694 sha256=f8ea1bd12170b5473c971c72fb856712918c6d5f5a0ba71944a2399d178ba799
  Stored in directory: /tmp/pip-ephem-wheel-cache-z4lf6o3_/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
 

In [55]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [56]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [57]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [58]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [59]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

This function wraps the call to *model.generate*

In [60]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [30]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Employees (
        EmployeeID int,
        Name varchar(255),
        Position varchar(255),
        Salary int
    );
    CREATE TABLE Departments (
        DepartmentID int,
        DepartmentName varchar(255)
    );
    CREATE TABLE EmployeeDepartment (
        EmployeeID int,
        DepartmentID int
    );

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Find all employees in the 'Engineering' department`:
    ```sql3
    """

In [31]:
sp_nl2sql = sp_nl2sql.format(question="Find all employees in the 'Engineering' department")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Employees (
        EmployeeID int,
        Name varchar(255),
        Position varchar(255),
        Salary int
    );
    CREATE TABLE Departments (
        DepartmentID int,
        DepartmentName varchar(255)
    );
    CREATE TABLE EmployeeDepartment (
        EmployeeID int,
        DepartmentID int
    );

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Find all employees in the 'Engineering' department`:
    ```sql3
    


In [32]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [33]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [34]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT Employees.EmployeeID, Employees.Name, Employees.Salary FROM Employees JOIN EmployeeDepartment ON Employees.EmployeeID = EmployeeDepartment.EmployeeID JOIN Departments ON EmployeeDepartment.DepartmentID = Departments.DepartmentID WHERE Departments.DepartmentName ILIKE '%Engineering%' ORDER BY Employees.EmployeeID NULLS LAST;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [35]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the database's structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Employees (
        EmployeeID int,
        Name varchar(255),
        Position varchar(255),
        Salary int
    );
    CREATE TABLE Departments (
        DepartmentID int,
        DepartmentName varchar(255)
    );
    CREATE TABLE EmployeeDepartment (
        EmployeeID int,
        DepartmentID int
    );

    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL queries:

    Question: Find the names of all employees in the 'Sales' department.
    SQL Query:
    ```sql
    SELECT e.Name
    FROM Employees e
    JOIN EmployeeDepartment ed ON e.EmployeeID = ed.EmployeeID
    JOIN Departments d ON ed.DepartmentID = d.DepartmentID
    WHERE d.DepartmentName = 'Sales';
    ```

    Question: List the departments and the number of employees in each.
    SQL Query:
    ```sql
    SELECT d.DepartmentName, COUNT(ed.EmployeeID) as NumberOfEmployees
    FROM Departments d
    LEFT JOIN EmployeeDepartment ed ON d.DepartmentID = ed.DepartmentID
    GROUP BY d.DepartmentName;
    ```

    ### Response
    Based on your instructions, here is the SQL query generated to answer the question
    `Return the name of the best paid employee`:
    ```sql3
    """


In [36]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the database's structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Employees (
        EmployeeID int,
        Name varchar(255),
        Position varchar(255),
        Salary int
    );
    CREATE TABLE Departments (
        DepartmentID int,
        DepartmentName varchar(255)
    );
    CREATE TABLE EmployeeDepartment (
        EmployeeID int,
        DepartmentID int
    );

    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL queries:
    
    Question: Find the names of all employees in the 'Sa

In [37]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [38]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, MAX(e.salary) AS max_salary
    FROM Employees e
    GROUP BY e.name
    ORDER BY max_salary DESC
    LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [39]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the database's structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Employees (
        EmployeeID int,
        Name varchar(255),
        Position varchar(255),
        Salary int
    );
    CREATE TABLE Departments (
        DepartmentID int,
        DepartmentName varchar(255)
    );
    CREATE TABLE EmployeeDepartment (
        EmployeeID int,
        DepartmentID int
    );

    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL queries:

    Question: Find the names of all employees in the 'Sales' department.
    SQL Query:
    ```sql
    SELECT e.Name
    FROM Employees e
    JOIN EmployeeDepartment ed ON e.EmployeeID = ed.EmployeeID
    JOIN Departments d ON ed.DepartmentID = d.DepartmentID
    WHERE d.DepartmentName = 'Sales';
    ```

    Question: List the departments and the number of employees in each.
    SQL Query:
    ```sql
    SELECT d.DepartmentName, COUNT(ed.EmployeeID) as NumberOfEmployees
    FROM Departments d
    LEFT JOIN EmployeeDepartment ed ON d.DepartmentID = ed.DepartmentID
    GROUP BY d.DepartmentName;
    ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return the name of the best paid employee`:
    ```sql3
    """


In [40]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the database's structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Employees (
        EmployeeID int,
        Name varchar(255),
        Position varchar(255),
        Salary int
    );
    CREATE TABLE Departments (
        DepartmentID int,
        DepartmentName varchar(255)
    );
    CREATE TABLE EmployeeDepartment (
        EmployeeID int,
        DepartmentID int
    );
    
    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL queries:
    
    Question: Find the names of all employees in the

In [41]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [42]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, MAX(e.salary) AS max_salary
    FROM Employees e
    GROUP BY e.name
    ORDER BY max_salary DESC
    LIMIT 1;


#Now the question in spanish.


In [43]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Dame el nombre del empleado mejor pagado")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the database's structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Employees (
        EmployeeID int,
        Name varchar(255),
        Position varchar(255),
        Salary int
    );
    CREATE TABLE Departments (
        DepartmentID int,
        DepartmentName varchar(255)
    );
    CREATE TABLE EmployeeDepartment (
        EmployeeID int,
        DepartmentID int
    );
    
    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL queries:
    
    Question: Find the names of all employees in the

In [44]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [45]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, MAX(e.salary) AS max_salary
    FROM Employees e
    GROUP BY e.name
    ORDER BY max_salary DESC
    LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [63]:
sp_nl2sql6 = """
    ### Instructions:
Detective, your task is to investigate and convert a question into an SQL query, based on the database schema provided. Adhere to these rules:
- **Diligently analyze the question and database schema word by word** to uncover the truth.
- **Refer to the sample clues in the ### Samples section to understand the database's structure.**

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Suspects (
        SuspectID int,
        SuspectName varchar(255),
        Crime varchar(255),
        SentenceLength int
    );
    CREATE TABLE Crimes (
        CrimeID int,
        CrimeName varchar(255)
    );
    CREATE TABLE SuspectCrimes (
        SuspectID int,
        CrimeID int
    );

    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL clues:

    Question: Uncover the suspects involved in the 'Robbery' crime.
    SQL Clue:
    ```sql
    SELECT s.SuspectName
    FROM Suspects s
    JOIN SuspectCrimes sc ON s.SuspectID = sc.SuspectID
    JOIN Crimes c ON sc.CrimeID = c.CrimeID
    WHERE c.CrimeName = 'Robbery';
    ```

    Question: Identify the crimes and the number of suspects involved in each.
    SQL Clue:
    ```sql
    SELECT c.CrimeName, COUNT(sc.SuspectID) as NumberOfSuspects
    FROM Crimes c
    LEFT JOIN SuspectCrimes sc ON c.CrimeID = sc.CrimeID
    GROUP BY c.CrimeName;
    ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return the name of the suspect with the longest sentence`:
    ```sql3
    """
sp_nl2sql6 = sp_nl2sql6.format(question="Return the name of the suspect with the longest sentence")
print(sp_nl2sql6)




    ### Instructions:
Detective, your task is to investigate and convert a question into an SQL query, based on the database schema provided. Adhere to these rules:
- **Diligently analyze the question and database schema word by word** to uncover the truth.
- **Refer to the sample clues in the ### Samples section to understand the database's structure.**

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Suspects (
        SuspectID int,
        SuspectName varchar(255),
        Crime varchar(255),
        SentenceLength int
    );
    CREATE TABLE Crimes (
        CrimeID int,
        CrimeName varchar(255)
    );
    CREATE TABLE SuspectCrimes (
        SuspectID int,
        CrimeID int
    );
    
    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL clues:
    
    Question: Uncover the suspects involved in 

In [65]:
input_sentences = tokenizer(sp_nl2sql6, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()


In [66]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT s.SuspectName
    FROM Suspects s
    JOIN SuspectCrimes sc ON s.SuspectID = sc.SuspectID
    JOIN Crimes c ON sc.CrimeID = c.CrimeID
    WHERE c.CrimeName = 'Robbery'
    ORDER BY s.SentenceLength DESC NULLS LAST
    LIMIT 1;


In [68]:
sp_nl2sql5 = """
    ### Instructions:
Greetings, Culinary Query Chef! Your challenge is to whip up an SQL query from the ingredients provided in the database schema. Adhere to these rules:
- **Carefully consider each word in the question and schema** to create a perfect query dish.
- **Consult the sample recipes in the ### Samples section for culinary inspiration.**

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Recipes (
        RecipeID int,
        DishName varchar(255),
        ChefName varchar(255),
        Rating int
    );
    CREATE TABLE Ingredients (
        IngredientID int,
        IngredientName varchar(255)
    );
    CREATE TABLE RecipeIngredients (
        RecipeID int,
        IngredientID int
    );

    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL recipes:

    Question: Find the chefs who have created dishes with 'Tomato'.
    SQL Recipe:
    ```sql
    SELECT r.ChefName
    FROM Recipes r
    JOIN RecipeIngredients ri ON r.RecipeID = ri.RecipeID
    JOIN Ingredients i ON ri.IngredientID = i.IngredientID
    WHERE i.IngredientName = 'Tomato';
    ```

    Question: List the dishes and the number of ingredients in each.
    SQL Recipe:
    ```sql
    SELECT r.DishName, COUNT(ri.IngredientID) as NumberOfIngredients
    FROM Recipes r
    LEFT JOIN RecipeIngredients ri ON r.RecipeID = ri.RecipeID
    GROUP BY r.DishName;
    ```

    ### Response
    Based on your instructions, here is the SQL query crafted to answer the question
    `Return the name of the highest-rated dish`:
    ```sql3
    """
sp_nl2sql5 = sp_nl2sql5.format(question="Return the name of the highest-rated dish")
print(sp_nl2sql5)



    ### Instructions:
Greetings, Culinary Query Chef! Your challenge is to whip up an SQL query from the ingredients provided in the database schema. Adhere to these rules:
- **Carefully consider each word in the question and schema** to create a perfect query dish.
- **Consult the sample recipes in the ### Samples section for culinary inspiration.**

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Recipes (
        RecipeID int,
        DishName varchar(255),
        ChefName varchar(255),
        Rating int
    );
    CREATE TABLE Ingredients (
        IngredientID int,
        IngredientName varchar(255)
    );
    CREATE TABLE RecipeIngredients (
        RecipeID int,
        IngredientID int
    );
    
    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL recipes:
    
    Question: Find the chefs who hav

In [69]:
input_sentences = tokenizer(sp_nl2sql5, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [70]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT r.DishName
    FROM Recipes r
    JOIN RecipeIngredients ri ON r.RecipeID = ri.RecipeID
    JOIN Ingredients i ON ri.IngredientID = i.IngredientID
    WHERE i.IngredientName = 'Tomato'
    GROUP BY r.DishName
    HAVING COUNT(ri.IngredientID) = (SELECT MAX(NumberOfIngredients) FROM (SELECT r.DishName, COUNT(ri.IngredmentID) AS NumberOfIngredients
    FROM Recipes r
    LEFT JOIN RecipeIngredients ri ON r.RecipeID = ri.RecipeID
    GROUP BY r.DishName) AS Subquery);


In [71]:
sp_nl2sql7 = """
    ### Instructions:
Greetings, Adventurer! Your quest is to transform a mystical question into an SQL query using the enchanted database schema. Adhere to these rules:
- **Thoroughly examine each word in the question and schema** to ensure your query is precise.
- **Refer to the magical samples in the ### Samples section to understand the enchanted structure.**

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Heroes (
        HeroID int,
        HeroName varchar(255),
        HeroClass varchar(255),
        PowerLevel int
    );
    CREATE TABLE Guilds (
        GuildID int,
        GuildName varchar(255)
    );
    CREATE TABLE HeroGuild (
        HeroID int,
        GuildID int
    );

    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL spells:

    Question: Identify the heroes in the 'Warriors' guild.
    SQL Spell:
    ```sql
    SELECT h.HeroName
    FROM Heroes h
    JOIN HeroGuild hg ON h.HeroID = hg.HeroID
    JOIN Guilds g ON hg.GuildID = g.GuildID
    WHERE g.GuildName = 'Warriors';
    ```

    Question: Count the number of heroes per guild.
    SQL Spell:
    ```sql
    SELECT g.GuildName, COUNT(hg.HeroID) as NumberOfHeroes
    FROM Guilds g
    LEFT JOIN HeroGuild hg ON g.GuildID = hg.GuildID
    GROUP BY g.GuildName;
    ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return the name of the hero with the highest power level`:
    ```sql3
    """
sp_nl2sql7 = sp_nl2sql7.format(question="Return the name of the hero with the highest power level")
print(sp_nl2sql7)




    ### Instructions:
Greetings, Adventurer! Your quest is to transform a mystical question into an SQL query using the enchanted database schema. Adhere to these rules:
- **Thoroughly examine each word in the question and schema** to ensure your query is precise.
- **Refer to the magical samples in the ### Samples section to understand the enchanted structure.**

    ### Input
    Generate an SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Heroes (
        HeroID int,
        HeroName varchar(255),
        HeroClass varchar(255),
        PowerLevel int
    );
    CREATE TABLE Guilds (
        GuildID int,
        GuildName varchar(255)
    );
    CREATE TABLE HeroGuild (
        HeroID int,
        GuildID int
    );
    
    ### Samples
    Given the schema provided, here are some sample questions and their corresponding SQL spells:
    
    Question: Identify the heroes in the 'Warriors' 

In [72]:
input_sentences = tokenizer(sp_nl2sql7, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()


In [73]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT Heroes.HeroName
    FROM Heroes
    ORDER BY Heroes.PowerLevel DESC
    LIMIT 1;


**Report**
* Version 5 envisions their data task as preparing a culinary masterpiece, utilizing a database of recipes and ingredients.

* Version 6 transforms the SQL query task into a detective's investigation, making the process exciting by framing it as uncovering clues and solving cases within a database of suspects and crimes.

* Version 7 makes the the SQL query task as a fantasy quest, making it an enchanting experience where users embark on an adventure through a database of heroes and guilds.

I learned how to use Secret keys instead of OpenAI